In [1]:
pip install tensorflow-recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 8.8 MB/s 
     |████████████████████████████████| 578.0 MB 13 kB/s 
     |████████████████████████████████| 1.7 MB 46.6 MB/s 
     |████████████████████████████████| 438 kB 58.9 MB/s 
     |████████████████████████████████| 5.9 MB 25.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstal

In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pprint

In [2]:
data = pd.read_csv('coursesData.csv')

In [3]:
data['id'] = data['id'].map(str)

In [4]:
ratings = tf.data.Dataset.from_tensor_slices(dict(data))
courses = tf.data.Dataset.from_tensor_slices(dict(data[['course']]))

In [5]:
for rate in ratings.take(5).as_numpy_iterator():
  pprint.pprint(rate)

{'course': b'BA in Anthropology', 'id': b'0'}
{'course': b'BA in Archeology', 'id': b'1'}
{'course': b'BA in Archeology and Geography ', 'id': b'2'}
{'course': b'BA in Archeology and History ', 'id': b'3'}
{'course': b'BA in Art and Design', 'id': b'4'}


In [6]:
ratings1 = ratings.map(lambda x:{
    'id':x['id'],
    'course':x['course'],
})
courses1 = courses.map(lambda x:x['course'])

In [7]:
class CoursesModel(tfrs.Model):
  def __init__(self,idModel:tf.keras.Sequential(),courseModel:tf.keras.Sequential(),task:tfrs.tasks.Retrieval()):
    super().__init__()
    self.idModel = idModel
    self.courseModel = courseModel
    self.task  = task

  def compute_loss(self,features,training=False)->tf.Tensor:
    id_embeddings = self.idModel(features['course'])
    course_embeddings = self.courseModel(features['course'])

    return self.task(id_embeddings,course_embeddings)



In [8]:
id_vocabulary = tf.keras.layers.StringLookup()
id_vocabulary.adapt(ratings1.map(lambda x:x['id']))
courses_vocabulary = tf.keras.layers.TextVectorization()
courses_vocabulary.adapt(courses1)

In [9]:
idModel = tf.keras.Sequential([
    courses_vocabulary,
    tf.keras.layers.Embedding(courses_vocabulary.vocabulary_size(),32),
    tf.keras.layers.GlobalAveragePooling1D()
    
])
courseModel = tf.keras.Sequential([
    courses_vocabulary,
    tf.keras.layers.Embedding(courses_vocabulary.vocabulary_size(),32),
    tf.keras.layers.GlobalAveragePooling1D()
])

In [10]:
task = tfrs.tasks.Retrieval(metrics = tfrs.metrics.FactorizedTopK(courses1.batch(10).map(courseModel)))

In [11]:
model = CoursesModel(idModel,courseModel,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [12]:
model.fit(ratings1.batch(10),epochs=300)

Epoch 1/300
12/12 [==============================] - 7s 83ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0357 - factorized_top_k/top_10_categorical_accuracy: 0.1161 - factorized_top_k/top_50_categorical_accuracy: 0.4821 - factorized_top_k/top_100_categorical_accuracy: 0.9732 - loss: 19.6788 - regularization_loss: 0.0000e+00 - total_loss: 19.6788
Epoch 2/300
12/12 [==============================] - 1s 71ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0804 - factorized_top_k/top_10_categorical_accuracy: 0.1696 - factorized_top_k/top_50_categorical_accuracy: 0.5446 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 19.4974 - regularization_loss: 0.0000e+00 - total_loss: 19.4974
Epoch 3/300
12/12 [==============================] - 1s 64ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accurac

In [13]:
index = tfrs.layers.factorized_top_k.BruteForce(model.idModel)
index.index_from_dataset(courses1.batch(20).map(lambda course:(course,model.courseModel(course))))

In [21]:
! pip install scann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.idModel)
scann_index.index_from_dataset(courses1.batch(20).map(lambda course:(course,model.courseModel(course))))


In [16]:
for i in range(0,112):
  course,titles = scann_index(np.array([data.iloc[i][1]]))
  print(f'course: {data.iloc[i][1]} \n Neighbours are {titles} \n values are {course}')
  print('\n \n')

course: BA in Anthropology 
 Neighbours are [[b'BSc in Electrical Engineering BSc in Mechanical Engineering'
  b'BSc in Chemical and Process Engineering BSc in Civil Engineering'
  b'Bachelor of Commerce in Human Resources Management '
  b'Bachelor of Commerce in Tourism and Hospitality Management '
  b'BA in Philosophy and Ethics ' b'BA in Heritage Management '
  b'BSc in Agricultural Engineering and Mechanization '
  b'BSc in Computer Engineering and Information Technology '
  b'BA in Film and Television ' b'MSc. in Applied Botany ']] 
 values are [[10.985929   8.42128    8.3187065  8.014904   7.785783   7.729642
   7.5023665  7.472683   7.251831   7.1570544]]

 

course: BA in Archeology 
 Neighbours are [[b'BA in Philosophy and Ethics '
  b'BSc in Computer Engineering and Information Technology '
  b'BSc in Electrical Engineering BSc in Mechanical Engineering'
  b'BA in Archeology and Geography ' b'BA in Archeology and History '
  b'BSc in Agricultural Engineering and Mechanization

In [17]:
for i in range(0,112):
  course,titles = index(np.array([data.iloc[i][1]]))
  print(f'course: {data.iloc[i][1]} \n Neighbours are {titles} \n values are {course}')
  print('\n \n')

course: BA in Anthropology 
 Neighbours are [[b'BSc in Electrical Engineering BSc in Mechanical Engineering'
  b'BSc in Chemical and Process Engineering BSc in Civil Engineering'
  b'Bachelor of Commerce in Human Resources Management '
  b'Bachelor of Commerce in Tourism and Hospitality Management '
  b'BA in Philosophy and Ethics ' b'BA in Heritage Management '
  b'BSc in Agricultural Engineering and Mechanization '
  b'BSc in Computer Engineering and Information Technology '
  b'BA in Film and Television ' b'MSc. in Applied Botany ']] 
 values are [[10.9859295  8.421281   8.3187065  8.014905   7.785783   7.7296424
   7.502367   7.472683   7.251831   7.1570544]]

 

course: BA in Archeology 
 Neighbours are [[b'BA in Philosophy and Ethics '
  b'BSc in Computer Engineering and Information Technology '
  b'BSc in Electrical Engineering BSc in Mechanical Engineering'
  b'BA in Archeology and Geography ' b'BA in Archeology and History '
  b'BSc in Agricultural Engineering and Mechanizatio

In [ ]:
index.save('model')

In [ ]:
!zip -r model.zip model

In [ ]:
np.array([data.iloc[1][1]])

array(['BA in Archeology'], dtype='<U16')